## EX03

In [147]:
import sqlite3
import pandas as pd

try:
    sqlite_connection = sqlite3.connect('../ex02/data/checking-logs.sqlite.sqlite')
except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)
print('УСПЕШНО')

УСПЕШНО


In [148]:
request = 'PRAGMA table_info(checker);'
pd.io.sql.read_sql(request, sqlite_connection)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


In [149]:
request = 'SELECT * FROM test;'
pd.io.sql.read_sql(request, sqlite_connection).head(10)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [150]:
request = 'SELECT * FROM deadlines;'
pd.io.sql.read_sql(request, sqlite_connection).head(10)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [151]:
request = \
"""
SELECT uid, min((JulianDay(datetime(deadlines, 'unixepoch')) - julianday(timestamp)) * 24) FROM checker
JOIN deadlines ON labname=labs and labs!='project1'
WHERE uid LIKE 'user_%'
;
"""
df_min = pd.io.sql.read_sql(request, sqlite_connection)
df_min

,uid,"min((JulianDay(datetime(deadlines, 'unixepoch')) - julianday(timestamp)) * 24)"
0,user_16,-206.894257


In [152]:
request = \
"""
SELECT uid, max((JulianDay(datetime(deadlines, 'unixepoch')) - julianday(timestamp)) * 24) FROM checker
JOIN deadlines ON labname=labs and labs!='project1'
WHERE checker.uid LIKE 'user_%'
;
"""
df_max = pd.io.sql.read_sql(request, sqlite_connection)
df_max

,uid,"max((JulianDay(datetime(deadlines, 'unixepoch')) - julianday(timestamp)) * 24)"
0,user_24,469.803918


In [153]:
request = \
"""
SELECT AVG((JulianDay(datetime(deadlines, 'unixepoch')) - julianday(timestamp)) * 24) FROM checker
JOIN deadlines ON labname=labs and labs!='project1'
WHERE checker.uid LIKE 'user_%'
;
"""
df_avg = pd.io.sql.read_sql(request, sqlite_connection)
df_avg

,"AVG((JulianDay(datetime(deadlines, 'unixepoch')) - julianday(timestamp)) * 24)"
0,68.426919


In [154]:
request = \
"""
SELECT * FROM checker
;
"""
res = pd.io.sql.read_sql(request, sqlite_connection)
res

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
...,...,...,...,...,...,...,...
3397,3397,ready,0,2020-05-21 20:19:06.872761,7,laba06s,user_1
3398,3398,checking,0,2020-05-21 20:22:41.785725,8,laba06s,user_1
3399,3399,ready,0,2020-05-21 20:22:41.877806,8,laba06s,user_1
3400,3400,checking,0,2020-05-21 20:37:00.129678,9,laba06s,user_1


### we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference

In [160]:
request = \
"""
SELECT test.uid,
       AVG(CAST( (JulianDay(test.first_view_ts) - 
                  JulianDay(DATETIME(deadlines.deadlines, 'unixepoch'))
                  ) * 24 AS Integer
                )
           ) AS delta, 
       pageviews
FROM test
LEFT JOIN deadlines ON test.labname=deadlines.labs
LEFT JOIN (SELECT uid, count(*) AS pageviews
           FROM pageviews
           GROUP BY uid) AS views ON test.uid=views.uid
WHERE labname != 'project1'
GROUP BY test.uid
;
"""
views_diff = pd.io.sql.read_sql(request, sqlite_connection)
views_diff

,uid,delta,pageviews
0,user_1,-304.4,28
1,user_10,-505.4,89
2,user_14,-261.0,143
3,user_17,-507.4,47
4,user_18,-57.0,3
5,user_19,-501.0,16
6,user_21,-307.0,10
7,user_25,9.0,179
8,user_28,31.0,149
9,user_3,-28.0,317


In [156]:
views_diff.corr()


,delta,pageviews
delta,1.000000,0.588922
pageviews,0.588922,1.000000


In [ ]:
sqlite_connection.close()